In [1]:
# 1.1: Install dependencies
# We use the same google-adk library as the original project
# !pip install google-adk uvicorn requests

# 1.2: Configure API Key
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(f"🔑 Authentication Error: {e}")

# 1.3: Import ADK Components
import asyncio
import json
import subprocess
import time
import uuid
import requests
from typing import List, Dict, Any
from google.genai import types

# Core ADK Imports
from google.adk.agents import (
    LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
)
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.tools import (
    AgentTool, FunctionTool, ToolContext, google_search
)
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a

# Configuration
retry_config = types.HttpRetryOptions(attempts=5, exp_base=2, initial_delay=1)
print("✅ ADK components imported successfully.")

✅ Setup and authentication complete.
✅ ADK components imported successfully.


In [2]:
# --- Specialist 1: DesignArchitectAgent ---
# Decides on the color palette and layout style.
design_architect_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="DesignArchitectAgent",
    instruction="""You are a UI/UX Design Lead.
    1. Analyze the user's request to determine the brand 'vibe' (e.g., corporate, playful, dark-mode).
    2. Define a color palette (Hex codes) and font pairing.
    3. Return a JSON object with keys: 'style_theme', 'primary_color', 'secondary_color', 'font_family'.""",
    output_key="design_specs"
)

# --- Specialist 2: ContentStrategistAgent ---
# Writes the actual text for the website.
content_strategist_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ContentStrategistAgent",
    instruction="""You are a Copywriter.
    1. Write a catchy 'Hero Headline' and a 'Sub-headline' based on the user request.
    2. Write a 50-word 'About Us' blurb.
    3. Return JSON with keys: 'hero_title', 'hero_subtitle', 'about_text'.""",
    output_key="content_assets"
)

# --- Specialist 3: MarketTrendAgent (Security/SEO) ---
# Checks for modern trends or requirements.
market_trend_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MarketTrendAgent",
    instruction="""You are an SEO & Trends Specialist.
    1. Use `Google Search` to find 'current web design trends 2025'.
    2. Identify one key feature to include (e.g., 'Glassmorphism', 'Large Typography').
    3. Return a short string describing this trend.""",
    tools=[google_search],
    output_key="trend_insights"
)

print("✅ Design & Strategy Team created.")

✅ Design & Strategy Team created.


In [3]:
# --- Specialist 4: FrontendCoderAgent ---
# The builder who writes the HTML/CSS.
frontend_coder_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    name="FrontendCoderAgent",
    instruction="""You are a Senior Frontend Developer.
    Based on the {blueprint}, {design_specs}, and {critique} (if any):
    1. Write a SINGLE valid HTML file containing embedded CSS.
    2. Use the specified colors: {design_specs}.
    3. Insert the content: {content_assets}.
    
    CRITICAL: Output ONLY the raw HTML code starting with <!DOCTYPE html>. Do not use markdown blocks.""",
    output_key="generated_code"
)

# --- Specialist 5: CodeReviewerAgent (Agent-as-a-Judge) ---
# The critic who checks for errors.
def check_html_validity(html_code: str) -> Dict[str, Any]:
    """MOCK: specific checks on the HTML structure."""
    print("Tool Call: Scanning HTML structure...")
    errors = []
    if "<!DOCTYPE html>" not in html_code:
        errors.append("Missing DOCTYPE declaration.")
    if "viewport" not in html_code:
        errors.append("Missing viewport meta tag for responsiveness.")
    
    if not errors:
        return {"status": "APPROVED", "quality_score": 100}
    else:
        return {"status": "FAILED", "errors": errors}

code_reviewer_agent = LlmAgent(
    # --- ADDED MODEL PARAMETER ---
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    # -----------------------------
    name="CodeReviewerAgent",
    instruction="""You are the Code Reviewer. 
    Review the code in {generated_code} using the blueprint and the previous critique.
    If the code meets all requirements, output ONLY the final HTML code (in {generated_code}) 
    and set the status to 'APPROVED'. If you output 'APPROVED', the loop will stop, and the code will be passed to the next agent.
    If the code needs changes, output a critique to {critique} and the code for the next iteration.""",
    output_key="critique"
)
print("✅ Engineering Team (Loop) created.")

✅ Engineering Team (Loop) created.


In [4]:
def request_publish_approval(
    tool_context: ToolContext,
    generated_code: str,
    site_name: str
) -> Dict[str, str]:
    """
    Pauses execution to show the user a preview and ask for publish confirmation.
    """
    # Resume Logic
    if tool_context.tool_confirmation:
        if tool_context.tool_confirmation.confirmed:
            print("Tool Call: ✅ User APPROVED publication.")
            return {"status": "APPROVED"}
        else:
            print("Tool Call: ❌ User REJECTED publication.")
            return {"status": "REJECTED"}

    # Pause Logic
    print(f"\n--- PREVIEW FOR {site_name} ---")
    print(generated_code[:500] + "... [truncated] ...")
    print("-------------------------------")
    
    tool_context.request_confirmation(
        hint=f"⚠️ Review the HTML code above. Publish '{site_name}'?",
        payload={"code_preview": generated_code[:200]}
    )
    return {"status": "PENDING_APPROVAL"}

print("✅ Preview & Approval Tool created.")

✅ Preview & Approval Tool created.


In [5]:
# 3.1: Create the A2A Server File
import textwrap

# This code defines the remote "Hosting" agent
a2a_server_code = textwrap.dedent('''
import os
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from typing import Dict

def upload_to_server(html_content: str, filename: str) -> Dict[str, str]:
    """MOCK: Simulates uploading the file to a production web server."""
    # In a real app, this would use FTP or an S3 API.
    # We will simulate it by writing to a local file.
    path = f"/tmp/{filename}"
    with open(path, "w") as f:
        f.write(html_content)
    print(f"A2A SERVER: 🚀 Website published to {path}")
    return {"status": "success", "live_url": f"https://agent-hosted.com/{filename}"}

retry_config = types.HttpRetryOptions(attempts=3)
hosting_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="HostingProviderAgent",
    instruction="You are a cloud hosting bot. You ONLY run the `upload_to_server` tool.",
    tools=[FunctionTool(upload_to_server)],
    output_key="deployment_result"
)

print("A2A SERVER: Exposing HostingProviderAgent...")
app = to_a2a(hosting_agent, port=8001)
''')

with open("/tmp/a2a_hosting_server.py", "w") as f:
    f.write(a2a_server_code)

# 3.2: Start the Server in Background
server_process = subprocess.Popen(
    ["uvicorn", "a2a_hosting_server:app", "--host", "localhost", "--port", "8001", "--log-level", "error"],
    cwd="/tmp",
    env={**os.environ},
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Wait for server readiness
print("🚀 Starting Hosting Agent on localhost:8001...")
time.sleep(5) # Give it a moment to boot
print("✅ Hosting Agent is running.")

🚀 Starting Hosting Agent on localhost:8001...
✅ Hosting Agent is running.


In [6]:
# ==============================================================================
# SECTION 4: Orchestration and Coordination (The Fix is here)
# ==============================================================================

# --- Step 1: Analysis ---
# NOTE: If you get a 'parent agent' error here, you MUST restart your kernel.
analysis_phase = ParallelAgent(
    name="AnalysisPhase",
    sub_agents=[design_architect_agent, content_strategist_agent, market_trend_agent]
)

# --- Step 2: Planning ---
blueprint_planner = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="SiteBlueprintAgent",
    instruction="""You are the Solutions Architect.
    Review the input:
    - Design: {design_specs}
    - Content: {content_assets}
    - Trends: {trend_insights}
    
    Synthesize this into a clear 'blueprint' prompt for the developer. 
    Ensure you mention the visual style and specific text to use.""",
    output_key="blueprint"
)

# --- Step 3: Construction (Loop) ---
coding_phase = LoopAgent(
    name="CodingPhase",
    sub_agents=[frontend_coder_agent, code_reviewer_agent],
    max_iterations=3
    # FIX: output_key REMOVED from LoopAgent
)

# --- Step 4: Approval ---
preview_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="UserPreviewAgent",
    instruction="""You are the Account Manager.
    The code is ready: {generated_code}.
    Call `request_publish_approval` to show it to the user.""",
    tools=[FunctionTool(request_publish_approval)],
    output_key="user_decision"
)

# --- Step 5: Deployment (Remote) ---

# Define the remote agent separately
hosting_provider_agent = RemoteA2aAgent(
    name="HostingProviderAgent",
    agent_card="http://localhost:8001/.well-known/agent-card.json"
)

# CRITICAL FIX: The instruction must explicitly guide the LLM to call the function
deployment_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="PublisherAgent",
    instruction="""You are the Release Engineer.
    If {user_decision} is 'APPROVED', you must call the remote tool `HostingProviderAgent` with its function `upload_file`.
    The arguments must be:
    - `file_content`: Use the state variable **{generated_code}** for this argument.
    - `filename`: Use the string value **'index.html'** for this argument.
    After the tool call, output the resulting URL directly as your final answer.""",
    tools=[
        AgentTool(hosting_provider_agent)
    ],
    output_key="final_url"
)

# --- ROOT COORDINATOR ---
web_builder_coordinator = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="WebBuilderCoordinator",
    instruction="""You are the AI Website Creator. Follow this process:
    1. ANALYZE: Call `AnalysisPhase` to gather design and content.
    2. PLAN: Call `SiteBlueprintAgent` to create the spec.
    3. BUILD: Call `CodingPhase` to generate and refine the code.
    4. PREVIEW: Call `UserPreviewAgent` to get human sign-off.
    5. PUBLISH: Call `PublisherAgent` to deploy the site.
    6. FINISH: Output the final live URL.""",
    tools=[
        AgentTool(analysis_phase),
        AgentTool(blueprint_planner),
        AgentTool(coding_phase),
        AgentTool(preview_agent),
        AgentTool(deployment_agent)
    ]
)

print("✅ WebBuilderCoordinator (Root Agent) assembled with latest fixes.")

✅ WebBuilderCoordinator (Root Agent) assembled with latest fixes.


/tmp/ipykernel_48/1507484956.py:49: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  hosting_provider_agent = RemoteA2aAgent(


In [7]:
# 5.1: Configure Runner
import os
import uuid
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.runners import Runner
from google.genai import types

# Reset the DB for a clean run
if os.path.exists("web_builder.db"):
    os.remove("web_builder.db")

session_service = DatabaseSessionService(db_url="sqlite:///web_builder.db")
memory_service = InMemoryMemoryService() 

app = App(
    name="AI_Web_Creator",
    root_agent=web_builder_coordinator,
    resumability_config=ResumabilityConfig(is_resumable=True),
    plugins=[LoggingPlugin()]
)

runner = Runner(
    app=app,
    session_service=session_service,
    memory_service=memory_service
)

# 5.2: Start the Workflow
session_id = f"build_{uuid.uuid4().hex[:8]}"
user_request = "Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents."

print(f"--- STARTING BUILD: '{user_request}' ---")

# --- 🛠️ FIX 1: EXPLICITLY CREATE THE SESSION ---
await session_service.create_session(
    app_name="AI_Web_Creator",
    user_id="client_01",
    session_id=session_id
)
print(f"✅ Session {session_id} created in database.")

# --- 🛠️ FIX 2: INITIALIZE THE MISSING VARIABLE ---
# We set 'critique' to a default string so the FrontendCoder doesn't crash on the first loop.
initial_state = {"critique": "No critique yet (first iteration)."}

invocation_id = None
async for event in runner.run_async(
    user_id="client_01",
    session_id=session_id,
    new_message=types.Content(role="user", parts=[types.Part(text=user_request)]),
    state_delta=initial_state  # <--- PASS THE INITIAL STATE HERE
):
    if event.invocation_id:
        invocation_id = event.invocation_id
        
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                print(f"[{event.author}] > {part.text[:100]}...") 
                
    # Detect HITL Pause
    if event.actions and event.actions.requested_tool_confirmations:
        print("\n⏸️ SYSTEM PAUSED: Waiting for User Approval...")
        break

print(f"--- PAUSED at Invocation ID: {invocation_id} ---")

--- STARTING BUILD: 'Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents.' ---


/tmp/ipykernel_48/2146376407.py:21: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


✅ Session build_154554bf created in database.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-6824f42b-ccc5-4fb5-be90-786c78087f1d
[logging_plugin]    Session ID: build_154554bf
[logging_plugin]    User ID: client_01
[logging_plugin]    App Name: AI_Web_Creator
[logging_plugin]    Root Agent: WebBuilderCoordinator
[logging_plugin]    User Content: text: 'Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-6824f42b-ccc5-4fb5-be90-786c78087f1d
[logging_plugin]    Starting Agent: WebBuilderCoordinator
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: WebBuilderCoordinator
[logging_plugin]    Invocation ID: e-6824f42b-ccc5-4fb5-be90-786c78087f1d
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    System Instruction: 'You are th

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: AnalysisPhase
[logging_plugin]    Token Usage - Input: 311, Output: 34
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: f25aff77-f9dd-4d42-a794-e1bc554ea3ff
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: AnalysisPhase
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['AnalysisPhase']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: AnalysisPhase
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-7026e427-0cda-47bf-8545-fcbf4e86b7c4
[logging_plugin]    Arguments: {'request': "Landing page for a futuristic coffee shop named 'Nebula Brew' with a dark theme and neon purple accents."}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-b3e6847e-af82-4f5f-aaac-b522dc4e6fb7
[logging_plugin]    Session 

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: SiteBlueprintAgent
[logging_plugin]    Token Usage - Input: 464, Output: 68
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: f3eb85bb-373c-44d4-a12a-640d4ca5a771
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: SiteBlueprintAgent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['SiteBlueprintAgent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: SiteBlueprintAgent
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-c4a5de40-0aa6-4eb0-8fe0-ed0be93215ae
[logging_plugin]    Arguments: {'request': "Create a landing page for 'Nebula Brew', a futuristic coffee shop. The design should incorporate a dark mode theme with neon purple accents. The page should include sections for: Hero with a tagline and call to action, About Us, M

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: CodingPhase
[logging_plugin]    Token Usage - Input: 2017, Output: 93
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: f6abda82-3171-44db-99b4-07c55a6a8902
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: CodingPhase
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['CodingPhase']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: CodingPhase
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-8e2a983d-ebf8-4618-9c66-046585b29377
[logging_plugin]    Arguments: {'request': "Generate the HTML and CSS for the 'Nebula Brew' landing page based on the provided blueprint. Ensure a dark mode theme with neon purple accents, 'Orbitron' for headings, and 'Montserrat' for body text. Include Hero, About Us, Menu Highlights (with 3-4 examp

In [8]:
# Assume all necessary imports are at the top of your notebook.
import os
import uuid
import asyncio
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.runners import Runner
from google.genai import types

# Reset the DB for a clean run
if os.path.exists("web_builder.db"):
    os.remove("web_builder.db")

session_service = DatabaseSessionService(db_url="sqlite:///web_builder.db")
memory_service = InMemoryMemoryService() 

app = App(
    name="AI_Web_Creator",
    root_agent=web_builder_coordinator,
    resumability_config=ResumabilityConfig(is_resumable=True),
    plugins=[LoggingPlugin()]
)

runner = Runner(
    app=app,
    session_service=session_service,
    memory_service=memory_service
)

# 5.2: Start the Workflow
session_id = f"build_{uuid.uuid4().hex[:8]}"
user_request = "Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents."

print(f"--- STARTING BUILD: '{user_request}' ---")

# FIX: EXPLICITLY CREATE THE SESSION & INITIALIZE MISSING VARIABLE
await session_service.create_session(
    app_name="AI_Web_Creator",
    user_id="client_01",
    session_id=session_id
)
print(f"✅ Session {session_id} created in database.")
initial_state = {"critique": "No critique yet (first iteration)."}

invocation_id = None
async for event in runner.run_async(
    user_id="client_01",
    session_id=session_id,
    new_message=types.Content(role="user", parts=[types.Part(text=user_request)]),
    state_delta=initial_state
):
    if event.invocation_id:
        invocation_id = event.invocation_id
        
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                print(f"[{event.author}] > {part.text[:100]}...") 
                
    # Detect HITL Pause
    if event.actions and event.actions.requested_tool_confirmations:
        print("\n⏸️ SYSTEM PAUSED: Waiting for User Approval...")
        break

print(f"--- PAUSED at Invocation ID: {invocation_id} ---")

--- STARTING BUILD: 'Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents.' ---
✅ Session build_583d9324 created in database.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-df995498-543c-4f31-aae7-e8f0175f8231
[logging_plugin]    Session ID: build_583d9324
[logging_plugin]    User ID: client_01
[logging_plugin]    App Name: AI_Web_Creator
[logging_plugin]    Root Agent: WebBuilderCoordinator
[logging_plugin]    User Content: text: 'Make a landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-df995498-543c-4f31-aae7-e8f0175f8231
[logging_plugin]    Starting Agent: WebBuilderCoordinator
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: WebBuilderCoordinator
[logging_plugin]    Invocation ID: e-df995498-543c-4f31-aae7-e8f0175f8231
[logging_plugin] 🧠 LLM REQUEST
[logging_plugi

/tmp/ipykernel_48/173527490.py:22: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: AnalysisPhase
[logging_plugin]    Token Usage - Input: 311, Output: 33
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: dd367dc4-e6bc-4754-9104-f7a26aa9cf15
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: AnalysisPhase
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['AnalysisPhase']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: AnalysisPhase
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-09190217-9cc2-4620-9a65-e58dddaf99ea
[logging_plugin]    Arguments: {'request': "Landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents."}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-c39eb754-e66d-4eac-89bf-c9059a2371f6
[logging_plugin]    Session ID: a030

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: SiteBlueprintAgent
[logging_plugin]    Token Usage - Input: 436, Output: 49
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: f853031d-5369-46a8-8845-df14ba5d8bea
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: SiteBlueprintAgent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['SiteBlueprintAgent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: SiteBlueprintAgent
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-175280c7-650d-49a2-afe5-02a7d51cae37
[logging_plugin]    Arguments: {'request': "Landing page for a futuristic coffee shop called 'Nebula Brew'. Dark theme, neon purple accents. Use 'Futuristic, Sci-Fi Gaming UI Aesthetics' design trend."}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: function_call: CodingPhase
[logging_plugin]    Token Usage - Input: 1785, Output: 721
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: c44aa36e-3cc7-4c1f-a2e1-b946ddda791c
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: function_call: CodingPhase
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['CodingPhase']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: CodingPhase
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Function Call ID: adk-f3a8e8b9-300a-4ce8-8e34-f3da43af66fa
[logging_plugin]    Arguments: {'request': 'Generate a landing page for \'Nebula Brew\' based on the provided blueprint. Incorporate the \'Futuristic, Sci-Fi Gaming UI Aesthetics\' design trend. \n\n**Blueprint Details:**\n\n*   **Overall Aesthetic & Brand Vibe:** Futuristic, cosmic, sophisticated, 

...

/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:379: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: PublisherAgent
[logging_plugin]    Content: function_call: HostingProviderAgent
[logging_plugin]    Token Usage - Input: 3247, Output: 3238
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: dd4479da-31ec-4eeb-960d-739946501bbb
[logging_plugin]    Author: PublisherAgent
[logging_plugin]    Content: function_call: HostingProviderAgent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['HostingProviderAgent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: HostingProviderAgent
[logging_plugin]    Agent: PublisherAgent
[logging_plugin]    Function Call ID: adk-0d46da0d-0ad2-4b72-9a2a-8d56c8db0fe5
[logging_plugin]    Arguments: {'request': 'Upload file \'index.html\' with the following HTML content:\n\'\'\'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Nebula Brew - Y

/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:239: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:309: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  part = part_converter(a

[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 778d23d5-c287-40f5-9611-2043f2be27a7
[logging_plugin]    Author: HostingProviderAgent
[logging_plugin]    Content: text: 'I have uploaded your file to the server. You can access it live at: https://agent-hosted.com/index.html'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: HostingProviderAgent
[logging_plugin]    Invocation ID: e-28bda54e-1d97-4b37-b893-1714dd7fe67e
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    Invocation ID: e-28bda54e-1d97-4b37-b893-1714dd7fe67e
[logging_plugin]    Final Agent: HostingProviderAgent
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: HostingProviderAgent
[logging_plugin]    Agent: PublisherAgent
[logging_plugin]    Function Call ID: adk-0d46da0d-0ad2-4b72-9a2a-8d56c8db0fe5
[logging_plugin]    Result: I have uploaded your file to the server. You can access it live at: https://agent-hosted.com/index.html


[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: WebBuilderCoordinator
[logging_plugin]    Content: text: 'The Nebula Brew landing page is live! You can view it at: https://agent-hosted.com/index.html'
[logging_plugin]    Token Usage - Input: 9018, Output: 25
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: ac1b6f0a-097b-4cc9-a30d-33f89c30d86d
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: text: 'The Nebula Brew landing page is live! You can view it at: https://agent-hosted.com/index.html'
[logging_plugin]    Final Response: True
[WebBuilderCoordinator] > The Nebula Brew landing page is live! You can view it at: https://agent-hosted.com/index.html...
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: d472a320-da00-49af-80ac-bc9e7b9ada21
[logging_plugin]    Author: WebBuilderCoordinator
[logging_plugin]    Content: None
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent

In [9]:
# Rerun the resume code you used successfully before.

print("\n👤 User Action: Clicking 'Approve' button...")

resume_message = types.Content(
    role="user",
    parts=[types.Part(text="The design looks great. Publish it!")]
)

# Use the invocation_id from your previous PAUSED output (assuming it's still defined)
final_output_text = "ERROR: Deployment agent failed to return final URL."

async for event in runner.run_async(
    user_id="client_01",
    session_id=session_id,
    new_message=resume_message,
    invocation_id=invocation_id 
):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                print(f"[{event.author}] > {part.text}")
                # Capture the final text output (from the coordinator)
                if event.author == "WebBuilderCoordinator":
                    final_output_text = part.text

# --- CRITICAL LOGGING ---
print("\n✅ Deployment agent has finished running.")
print(f"--- FINAL COORDINATOR OUTPUT: {final_output_text} ---")
# ------------------------


👤 User Action: Clicking 'Approve' button...
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-df995498-543c-4f31-aae7-e8f0175f8231
[logging_plugin]    Session ID: build_583d9324
[logging_plugin]    User ID: client_01
[logging_plugin]    App Name: AI_Web_Creator
[logging_plugin]    Root Agent: WebBuilderCoordinator
[logging_plugin]    User Content: text: 'The design looks great. Publish it!'

✅ Deployment agent has finished running.
--- FINAL COORDINATOR OUTPUT: ERROR: Deployment agent failed to return final URL. ---


/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


In [10]:
print("\n👤 User Action: Clicking 'Approve' button...")

resume_message = types.Content(
    role="user",
    parts=[types.Part(text="The design looks great. Publish it!")]
)

final_output_text = "ERROR: Deployment agent failed to return final URL."

async for event in runner.run_async(
    user_id="client_01",
    session_id=session_id,
    new_message=resume_message,
    invocation_id=invocation_id 
):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                print(f"[{event.author}] > {part.text}")
                # Capture the final text output (from the coordinator)
                if event.author == "WebBuilderCoordinator":
                    final_output_text = part.text

print("\n✅ Deployment agent has finished running.")
print(f"--- FINAL COORDINATOR OUTPUT: {final_output_text} ---")


👤 User Action: Clicking 'Approve' button...
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-df995498-543c-4f31-aae7-e8f0175f8231
[logging_plugin]    Session ID: build_583d9324
[logging_plugin]    User ID: client_01
[logging_plugin]    App Name: AI_Web_Creator
[logging_plugin]    Root Agent: WebBuilderCoordinator
[logging_plugin]    User Content: text: 'The design looks great. Publish it!'

✅ Deployment agent has finished running.
--- FINAL COORDINATOR OUTPUT: ERROR: Deployment agent failed to return final URL. ---


In [11]:
# Final Verification Code
try:
    with open("/tmp/index.html", "r") as f:
        print("\n📄 DEPLOYED FILE CONTENT (/tmp/index.html):")
        # Print a snippet of the HTML to confirm success
        print(f.read()[:500]) 
except FileNotFoundError:
    print("❌ File not found. Deployment failed.")

# Clean up the A2A server
if 'server_process' in globals():
    try:
        server_process.terminate()
        print("✅ Local A2A server (localhost:8001) stopped.")
    except Exception as e:
        print(f"Server termination error: {e}")


📄 DEPLOYED FILE CONTENT (/tmp/index.html):
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Nebula Brew - Your Cosmic Coffee Escape</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&family=Orbitron:wght@700;900&display=swap" rel="stylesheet">

    <style>
        /* -
✅ Local A2A server (localhost:8001) stopped.


In [12]:
# Final Verification Code (Rerun this after Step 2 completes)
try:
    with open("/tmp/index.html", "r") as f:
        print("\n📄 DEPLOYED FILE CONTENT (/tmp/index.html):")
        print(f.read()[:500])
except FileNotFoundError:
    print("❌ File not found. Deployment failed.")

# Clean up
if 'server_process' in globals():
    server_process.terminate()
    print("✅ Local A2A server (localhost:8001) stopped.")


📄 DEPLOYED FILE CONTENT (/tmp/index.html):
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Nebula Brew - Your Cosmic Coffee Escape</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&family=Orbitron:wght@700;900&display=swap" rel="stylesheet">

    <style>
        /* -
✅ Local A2A server (localhost:8001) stopped.


In [13]:
# The Verification Code
try:
    with open("/tmp/index.html", "r") as f:
        print("\n📄 DEPLOYED FILE CONTENT (/tmp/index.html):")
        print(f.read()[:500])
except FileNotFoundError:
    print("❌ File not found. Deployment failed.")

# --- FINAL CLEANUP (DO THIS LAST!) ---
if 'server_process' in globals():
    server_process.terminate()
    print("✅ Local A2A server (localhost:8001) stopped.")


📄 DEPLOYED FILE CONTENT (/tmp/index.html):
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Nebula Brew - Your Cosmic Coffee Escape</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&family=Orbitron:wght@700;900&display=swap" rel="stylesheet">

    <style>
        /* -
✅ Local A2A server (localhost:8001) stopped.
